In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import streamlit as st
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
import plotly.graph_objects as go
import plotly.express as px
from scipy import signal
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score
import os

# Importation et Netoyage des données

In [20]:
# Charger les données

data = pd.read_csv('data_anonymized.csv', encoding='utf-8')
    
# Afficher les premières lignes pour comprendre la structure
print("Aperçu des données:")
print(data.head())


Aperçu des données:
   Batch name      Step  Niveau de la cuve  Température fond de cuve  \
0  Batch_2772  réaction               4.32                     28.48   
1  Batch_2772  réaction             695.30                     23.00   
2  Batch_2772  réaction             661.50                     21.79   
3  Batch_2772  réaction             673.70                     22.24   
4  Batch_2772  réaction             686.00                     24.65   

   Température haut de colonne  Température réacteur  Vitesse d'agitation  \
0                        31.19                 33.70                 0.13   
1                        29.68                 32.20                60.84   
2                        29.38                 31.30                59.02   
3                        29.08                 31.45                60.47   
4                        28.77                 31.90                60.99   

                  Time  
0  2023-01-01 00:00:00  
1  2023-01-01 00:05:00  
2  2023-0

In [21]:
# Informations générales sur le dataset
print("\nInformations sur le dataset:")
print(f"Nombre de lignes: {data.shape[0]}, Nombre de colonnes: {data.shape[1]}")
    


Informations sur le dataset:
Nombre de lignes: 264300, Nombre de colonnes: 8


In [22]:
data.columns

Index(['Batch name', 'Step', 'Niveau de la cuve', 'Température fond de cuve',
       'Température haut de colonne', 'Température réacteur',
       'Vitesse d'agitation', 'Time'],
      dtype='object')

In [24]:
# Vérifier les types de données
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 264300 entries, 0 to 264299
Data columns (total 8 columns):
 #   Column                       Non-Null Count   Dtype  
---  ------                       --------------   -----  
 0   Batch name                   264300 non-null  object 
 1   Step                         264300 non-null  object 
 2   Niveau de la cuve            264300 non-null  float64
 3   Température fond de cuve     264300 non-null  float64
 4   Température haut de colonne  264300 non-null  float64
 5   Température réacteur         264300 non-null  float64
 6   Vitesse d'agitation          264300 non-null  float64
 7   Time                         264300 non-null  object 
dtypes: float64(5), object(3)
memory usage: 16.1+ MB


In [25]:
import skimpy as sp

stats = sp.skim(data)  # Obtenir les statistiques
print(stats)  # Afficher les statistiques

╭──────────────────────────────────────────────── skimpy summary ─────────────────────────────────────────────────╮
│          Data Summary                Data Types                                                                 │
│ ┏━━━━━━━━━━━━━━━━━━━┳━━━━━━━━┓ ┏━━━━━━━━━━━━━┳━━━━━━━┓                                                          │
│ ┃ dataframe         ┃ Values ┃ ┃ Column Type ┃ Count ┃                                                          │
│ ┡━━━━━━━━━━━━━━━━━━━╇━━━━━━━━┩ ┡━━━━━━━━━━━━━╇━━━━━━━┩                                                          │
│ │ Number of rows    │ 264300 │ │ float64     │ 5     │                                                          │
│ │ Number of columns │ 8      │ │ string      │ 3     │                                                          │
│ └───────────────────┴────────┘ └─────────────┴───────┘                                                          │
│                                                     number                                                      │
│ ┏━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━┳━━━━━━━┳━━━━━━━━┳━━━━━━━━┳━━━━━━━━━┳━━━━━━━━┳━━━━━━━┳━━━━━━━┳━━━━━━━┳━━━━━━━━┓  │
│ ┃ column_name            ┃ NA  ┃ NA %  ┃ mean   ┃ sd     ┃ p0      ┃ p25    ┃ p50   ┃ p75   ┃ p100  ┃ hist   ┃  │
│ ┡━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━╇━━━━━━━╇━━━━━━━━╇━━━━━━━━╇━━━━━━━━━╇━━━━━━━━╇━━━━━━━╇━━━━━━━╇━━━━━━━╇━━━━━━━━┩  │
│ │ Niveau de la cuve      │   0 │     0 │   2202 │   2370 │  -279.3 │   4.32 │  1161 │  3792 │  8610 │ ▇▂▃▁▂  │  │
│ │ Température fond de    │   0 │     0 │  48.26 │  22.94 │       0 │  26.45 │ 45.38 │ 71.68 │   125 │ ▂▇▅▆▁  │  │
│ │ cuve                   │     │       │        │        │         │        │       │       │       │        │  │
│ │ Température haut de    │   0 │     0 │  40.14 │  22.84 │       0 │  24.18 │ 29.08 │ 55.14 │ 104.6 │  ▇▁▁▃  │  │
│ │ colonne                │     │       │        │        │         │        │       │       │       │        │  │
│ │ Température réacteur   │   0 │     0 │  49.89 │  22.61 │   -4.97 │  28.74 │ 49.14 │ 72.85 │ 185.1 │  ▃▇▇   │  │
│ │ Vitesse d'agitation    │   0 │     0 │  49.31 │  48.15 │       0 │   0.13 │ 59.02 │ 99.68 │ 110.3 │ ▇▁ ▂ ▇ │  │
│ └────────────────────────┴─────┴───────┴────────┴────────┴─────────┴────────┴───────┴───────┴───────┴────────┘  │
│                                                     string                                                      │
│ ┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━┓  │
│ ┃ column_name               ┃ NA      ┃ NA %       ┃ words per row                ┃ total words              ┃  │
│ ┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━┩  │
│ │ Batch name                │       0 │          0 │                            1 │                   264300 │  │
│ │ Step                      │       0 │          0 │                          1.9 │                   497903 │  │
│ │ Time                      │       0 │          0 │                            2 │                   528600 │  │
│ └───────────────────────────┴─────────┴────────────┴──────────────────────────────┴──────────────────────────┘  │
╰────────────────────────────────────────────────────── End ──────────────────────────────────────────────────────╯

None


In [26]:
# Statistiques descriptives
print("\nStatistiques descriptives:")
print(data.describe())


Statistiques descriptives:
       Niveau de la cuve  Température fond de cuve  \
count      264300.000000             264300.000000   
mean         2202.266429                 48.257354   
std          2370.088550                 22.936844   
min          -279.300000                  0.000000   
25%             4.320000                 26.450000   
50%          1161.000000                 45.380000   
75%          3792.000000                 71.680000   
max          8610.000000                125.000000   

       Température haut de colonne  Température réacteur  Vitesse d'agitation  
count                264300.000000         264300.000000        264300.000000  
mean                     40.142661             49.887688            49.305279  
std                      22.839926             22.605205            48.153045  
min                       0.000000             -4.970000             0.000000  
25%                      24.180000             28.740000             0.130000  
50%  

In [ ]:
#Gestion des valeurs manquantes
data['Température fond de cuve'].fillna(data['Température fond de cuve'].median(), inplace=True)
data['Température haut de colonne'].fillna(data['Température haut de colonne'].median(), inplace=True)
#data['Vitesse d'agitation'].fillna(data['Vitesse d'agitation'].median(), inplace=True)

In [28]:
# Vérifier les valeurs manquantes
print(data.isnull().sum())

Batch name                     0
Step                           0
Niveau de la cuve              0
Température fond de cuve       0
Température haut de colonne    0
Température réacteur           0
Vitesse d'agitation            0
Time                           0
dtype: int64


In [29]:
import skimpy as sp

stats = sp.skim(data)  # Obtenir les statistiques
print(stats)  # Afficher les statistiques

╭──────────────────────────────────────────────── skimpy summary ─────────────────────────────────────────────────╮
│          Data Summary                Data Types                                                                 │
│ ┏━━━━━━━━━━━━━━━━━━━┳━━━━━━━━┓ ┏━━━━━━━━━━━━━┳━━━━━━━┓                                                          │
│ ┃ dataframe         ┃ Values ┃ ┃ Column Type ┃ Count ┃                                                          │
│ ┡━━━━━━━━━━━━━━━━━━━╇━━━━━━━━┩ ┡━━━━━━━━━━━━━╇━━━━━━━┩                                                          │
│ │ Number of rows    │ 264300 │ │ float64     │ 5     │                                                          │
│ │ Number of columns │ 8      │ │ string      │ 3     │                                                          │
│ └───────────────────┴────────┘ └─────────────┴───────┘                                                          │
│                                                     number                                                      │
│ ┏━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━┳━━━━━━━┳━━━━━━━━┳━━━━━━━━┳━━━━━━━━━┳━━━━━━━━┳━━━━━━━┳━━━━━━━┳━━━━━━━┳━━━━━━━━┓  │
│ ┃ column_name            ┃ NA  ┃ NA %  ┃ mean   ┃ sd     ┃ p0      ┃ p25    ┃ p50   ┃ p75   ┃ p100  ┃ hist   ┃  │
│ ┡━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━╇━━━━━━━╇━━━━━━━━╇━━━━━━━━╇━━━━━━━━━╇━━━━━━━━╇━━━━━━━╇━━━━━━━╇━━━━━━━╇━━━━━━━━┩  │
│ │ Niveau de la cuve      │   0 │     0 │   2202 │   2370 │  -279.3 │   4.32 │  1161 │  3792 │  8610 │ ▇▂▃▁▂  │  │
│ │ Température fond de    │   0 │     0 │  48.26 │  22.94 │       0 │  26.45 │ 45.38 │ 71.68 │   125 │ ▂▇▅▆▁  │  │
│ │ cuve                   │     │       │        │        │         │        │       │       │       │        │  │
│ │ Température haut de    │   0 │     0 │  40.14 │  22.84 │       0 │  24.18 │ 29.08 │ 55.14 │ 104.6 │  ▇▁▁▃  │  │
│ │ colonne                │     │       │        │        │         │        │       │       │       │        │  │
│ │ Température réacteur   │   0 │     0 │  49.89 │  22.61 │   -4.97 │  28.74 │ 49.14 │ 72.85 │ 185.1 │  ▃▇▇   │  │
│ │ Vitesse d'agitation    │   0 │     0 │  49.31 │  48.15 │       0 │   0.13 │ 59.02 │ 99.68 │ 110.3 │ ▇▁ ▂ ▇ │  │
│ └────────────────────────┴─────┴───────┴────────┴────────┴─────────┴────────┴───────┴───────┴───────┴────────┘  │
│                                                     string                                                      │
│ ┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━┓  │
│ ┃ column_name               ┃ NA      ┃ NA %       ┃ words per row                ┃ total words              ┃  │
│ ┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━┩  │
│ │ Batch name                │       0 │          0 │                            1 │                   264300 │  │
│ │ Step                      │       0 │          0 │                          1.9 │                   497903 │  │
│ │ Time                      │       0 │          0 │                            2 │                   528600 │  │
│ └───────────────────────────┴─────────┴────────────┴──────────────────────────────┴──────────────────────────┘  │
╰────────────────────────────────────────────────────── End ──────────────────────────────────────────────────────╯

None


In [30]:
# Vérifier le nombre de lots uniques
print("\nNombre de lots uniques:", data['Batch name'].nunique())


Nombre de lots uniques: 264


In [31]:
# Vérifier les différentes étapes
print("\nÉtapes de production uniques:", data['Step'].unique())


Étapes de production uniques: ['réaction' 'traitement acido-basique' 'traitement thermique']


# Analyse exploratoire des données

In [ ]:
import streamlit as st
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
import os

# Configuration de la page
st.set_page_config(
    page_title="Analyse des procédés Sanofi",
    page_icon="💊",
    layout="wide"
)

# Titre et description
st.title("Analyse des Procédés de Production Sanofi")
st.markdown("""
Cette application permet de visualiser et analyser les données de capteurs 
pour les lots de production pharmaceutique. Elle aide à identifier 
les déviations de procédés en étudiant l'évolution de la température et d'autres paramètres.
""")

# Fonction pour charger les données
@st.cache_data
def load_data(file_path):
    try:
        data = pd.read_csv(file_path)
        # Conversion de la colonne Time en datetime si nécessaire
        if 'Time' in data.columns:
            data['Time'] = pd.to_datetime(data['Time'])
        return data
    except Exception as e:
        st.error(f"Erreur lors du chargement des données: {e}")
        return None

# Sidebar pour le chargement des données et les options
with st.sidebar:
    st.header("Configuration")
    
    # Upload des données
    uploaded_file = st.file_uploader("Charger le fichier CSV des données", type=['csv'])
    
    if uploaded_file is not None:
        data = load_data(uploaded_file)
    else:
        st.info("Veuillez charger un fichier CSV pour commencer l'analyse.")
        # Utilisation de données factices pour la démonstration
        st.markdown("### Mode démo")
        if st.checkbox("Utiliser des données de démonstration"):
            # Création de données factices basées sur la description
            np.random.seed(42)
            n_batches = 10
            n_observations = 1000
            
            demo_data = []
            steps = ["Préparation", "Réaction", "Purification"]
            
            for batch_idx in range(1, n_batches + 1):
                batch_name = f"Batch_{batch_idx}"
                
                for step_idx, step in enumerate(steps):
                    # Création de courbes de température avec variations
                    base_temp_cuve = 20 + step_idx * 30 + np.random.normal(0, 2, n_observations) 
                    base_temp_colonne = 15 + step_idx * 25 + np.random.normal(0, 1.5, n_observations)
                    base_temp_reacteur = 25 + step_idx * 35 + np.random.normal(0, 2.5, n_observations)
                    
                    # Ajout d'une tendance (montée puis stabilisation)
                    trend = np.zeros(n_observations)
                    rise_point = int(n_observations * 0.2)
                    stabilize_point = int(n_observations * 0.7)
                    
                    trend[:rise_point] = np.linspace(0, 15, rise_point)
                    trend[rise_point:stabilize_point] = np.linspace(15, 20, stabilize_point - rise_point)
                    trend[stabilize_point:] = 20
                    
                    # Application de la tendance
                    temp_cuve = base_temp_cuve + trend
                    temp_colonne = base_temp_colonne + trend * 0.8
                    temp_reacteur = base_temp_reacteur + trend * 1.2
                    
                    # Création des autres mesures
                    niveau_cuve = 50 + step_idx * 20 + np.cumsum(np.random.normal(0, 0.1, n_observations))
                    vitesse_agitation = 100 + step_idx * 50 + np.random.normal(0, 5, n_observations)
                    
                    # Timestamps
                    start_time = pd.Timestamp('2023-01-01') + pd.Timedelta(days=batch_idx-1)
                    timestamps = [start_time + pd.Timedelta(minutes=i*5) for i in range(n_observations)]
                    
                    for i in range(n_observations):
                        demo_data.append({
                            'Batch name': batch_name,
                            'Step': step,
                            'Niveau de la cuve': niveau_cuve[i],
                            'Température fond de cuve': temp_cuve[i],
                            'Température haut de colonne': temp_colonne[i],
                            'Température réacteur': temp_reacteur[i],
                            'Vitesse d\'agitation': vitesse_agitation[i],
                            'Time': timestamps[i]
                        })
            
            data = pd.DataFrame(demo_data)
            st.success("Données de démonstration chargées!")

# Corps principal de l'application
if 'data' in locals() and data is not None:
    st.header("Exploration des Données")
    
    # Statistiques de base
    st.subheader("Résumé des Données")
    col1, col2 = st.columns(2)
    with col1:
        st.write(f"Nombre total d'observations: {data.shape[0]}")
        st.write(f"Nombre de lots: {data['Batch name'].nunique()}")
    with col2:
        st.write(f"Étapes du procédé: {', '.join(data['Step'].unique())}")
        
    # Aperçu des données
    if st.checkbox("Afficher l'aperçu des données"):
        st.dataframe(data.head())

    # Section de sélection des lots
    st.header("Visualisation des Lots")
    
    # Onglets pour les différentes visualisations
    tabs = st.tabs(["Visualisation individuelle", "Superposition (Batch Overlay)", "Analyse comparative"])
    
    with tabs[0]:
        st.subheader("Visualisation d'un Lot Individuel")
        
        # Sélection du lot et de l'étape
        col1, col2 = st.columns(2)
        with col1:
            selected_batch = st.selectbox("Sélectionner un lot", options=sorted(data['Batch name'].unique()), key="vis_batch")
        with col2:
            selected_step = st.selectbox("Sélectionner une étape (optionnel)", 
                                       options=["Toutes les étapes"] + sorted(data['Step'].unique()), key="vis_step")
        
        # Filtrage des données
        if selected_step == "Toutes les étapes":
            filtered_data = data[data['Batch name'] == selected_batch]
        else:
            filtered_data = data[(data['Batch name'] == selected_batch) & (data['Step'] == selected_step)]
            
        if not filtered_data.empty:
            # Sélection des paramètres à visualiser
            params = st.multiselect(
                "Sélectionner les paramètres à visualiser",
                options=[col for col in data.columns if col not in ['Batch name', 'Step', 'Time']],
                default=['Température fond de cuve', 'Température haut de colonne', 'Température réacteur'],
                key="vis_params"
            )
            
            if params:
                # Création du graphique
                fig, ax = plt.figure(figsize=(10, 6)), plt.gca()
                
                for param in params:
                    if 'Time' in filtered_data.columns:
                        ax.plot(filtered_data['Time'], filtered_data[param], label=param)
                    else:
                        ax.plot(filtered_data.index, filtered_data[param], label=param)
                
                ax.set_title(f"Paramètres pour {selected_batch}" + (f" - {selected_step}" if selected_step != "Toutes les étapes" else ""))
                ax.set_xlabel("Temps / Index")
                ax.set_ylabel("Valeur")
                ax.legend(title="Paramètres")
                plt.tight_layout()
                
                st.pyplot(fig)
                
                # Option pour télécharger les données filtrées
                csv = filtered_data.to_csv(index=False)
                st.download_button(
                    label="Télécharger les données filtrées",
                    data=csv,
                    file_name=f"{selected_batch}_{selected_step.replace(' ', '_')}.csv",
                    mime='text/csv',
                )
            else:
                st.warning("Veuillez sélectionner au moins un paramètre à visualiser.")
        else:
            st.warning("Aucune donnée disponible pour ce lot et cette étape.")
    
    with tabs[1]:
        st.subheader("Superposition des Lots (Batch Overlay)")
        
        # Sélection des lots pour la superposition
        col1, col2 = st.columns(2)
        with col1:
            selected_batches = st.multiselect(
                "Sélectionner les lots à superposer",
                options=sorted(data['Batch name'].unique()),
                default=sorted(data['Batch name'].unique())[:2] if len(data['Batch name'].unique()) >= 2 else [],
                key="overlay_batches"
            )
        with col2:
            overlay_step = st.selectbox("Étape pour la superposition", 
                                     options=sorted(data['Step'].unique()),
                                     key="overlay_step")
        
        if selected_batches and overlay_step:
            # Sélection du paramètre à visualiser
            overlay_param = st.selectbox(
                "Paramètre à superposer",
                options=[col for col in data.columns if col not in ['Batch name', 'Step', 'Time']],
                index=data.columns.get_loc("Température fond de cuve") - 2 if "Température fond de cuve" in data.columns else 0,
                key="overlay_param"
            )
            
            # Superposition simple sans alignement complexe
            fig, ax = plt.figure(figsize=(10, 6)), plt.gca()
            
            # Afficher chaque lot
            for batch in selected_batches:
                batch_data = data[(data['Batch name'] == batch) & (data['Step'] == overlay_step)]
                if not batch_data.empty:
                    # Normalisation simple des index (0-100%)
                    batch_data = batch_data.reset_index(drop=True)
                    x_norm = np.linspace(0, 100, len(batch_data))
                    ax.plot(x_norm, batch_data[overlay_param], label=batch)
            
            ax.set_title(f"Superposition de {overlay_param} - {overlay_step}")
            ax.set_xlabel("Progression normalisée (%)")
            ax.set_ylabel(overlay_param)
            ax.legend(title="Lots")
            plt.tight_layout()
            
            st.pyplot(fig)
            
            # Option simple pour ajuster les données
            st.subheader("Filtrage des données")
            filter_start, filter_end = st.slider(
                "Filtrer les données (% de progression)",
                0, 100, (0, 100),
                step=5,
                key="filter_slider"
            )
            
            if filter_start > 0 or filter_end < 100:
                st.info(f"Filtrage appliqué: {filter_start}% à {filter_end}% de la progression")
                
                # Afficher les données filtrées
                fig, ax = plt.figure(figsize=(10, 6)), plt.gca()
                
                for batch in selected_batches:
                    batch_data = data[(data['Batch name'] == batch) & (data['Step'] == overlay_step)]
                    if not batch_data.empty:
                        batch_data = batch_data.reset_index(drop=True)
                        x_norm = np.linspace(0, 100, len(batch_data))
                        
                        # Appliquer le filtre
                        mask = (x_norm >= filter_start) & (x_norm <= filter_end)
                        ax.plot(x_norm[mask], batch_data[overlay_param].iloc[mask], label=batch)
                
                ax.set_title(f"Superposition filtrée de {overlay_param} - {overlay_step}")
                ax.set_xlabel(f"Progression normalisée (%) - Filtré de {filter_start}% à {filter_end}%")
                ax.set_ylabel(overlay_param)
                ax.legend(title="Lots")
                plt.tight_layout()
                
                st.pyplot(fig)
                
                # Préparation des données pour téléchargement
                filtered_data_dict = {}
                for batch in selected_batches:
                    batch_data = data[(data['Batch name'] == batch) & (data['Step'] == overlay_step)]
                    if not batch_data.empty:
                        batch_data = batch_data.reset_index(drop=True)
                        x_norm = np.linspace(0, 100, len(batch_data))
                        mask = (x_norm >= filter_start) & (x_norm <= filter_end)
                        filtered_data_dict[batch] = batch_data[overlay_param].iloc[mask].reset_index(drop=True)
                
                if filtered_data_dict:
                    # Transformer en DataFrame
                    max_len = max([len(series) for series in filtered_data_dict.values()])
                    for batch, series in filtered_data_dict.items():
                        if len(series) < max_len:
                            # Padding avec NaN
                            filtered_data_dict[batch] = pd.Series(list(series) + [np.nan] * (max_len - len(series)))
                    
                    filtered_df = pd.DataFrame(filtered_data_dict)
                    
                    # Option de téléchargement
                    csv = filtered_df.to_csv(index=True)
                    st.download_button(
                        label="Télécharger les données filtrées",
                        data=csv,
                        file_name=f"filtered_overlay_{overlay_param}_{overlay_step}.csv",
                        mime='text/csv',
                    )
            
        else:
            st.warning("Veuillez sélectionner au moins un lot et une étape pour la superposition.")
    
    with tabs[2]:
        st.subheader("Analyse Comparative et Détection des Déviations")
        
        # Sélectionner un lot de référence (idéal) et un lot à comparer
        col1, col2 = st.columns(2)
        with col1:
            ideal_batch = st.selectbox("Lot de référence (idéal)", 
                                     options=sorted(data['Batch name'].unique()),
                                     key="ideal_batch")
        with col2:
            compare_batch = st.selectbox("Lot à comparer", 
                                      options=[b for b in sorted(data['Batch name'].unique()) if b != ideal_batch],
                                      key="compare_batch")
        
        # Sélectionner l'étape à comparer
        compare_step = st.selectbox("Étape à comparer", 
                                  options=sorted(data['Step'].unique()),
                                  key="compare_step")
        
        if ideal_batch and compare_batch and compare_step:
            # Filtrer les données
            ideal_data = data[(data['Batch name'] == ideal_batch) & (data['Step'] == compare_step)]
            compare_data = data[(data['Batch name'] == compare_batch) & (data['Step'] == compare_step)]
            
            if not ideal_data.empty and not compare_data.empty:
                # Sélectionner les paramètres à comparer
                compare_params = st.multiselect(
                    "Paramètres à comparer",
                    options=[col for col in data.columns if col not in ['Batch name', 'Step', 'Time']],
                    default=['Température fond de cuve', 'Température haut de colonne'],
                    key="compare_params"
                )
                
                if compare_params:
                    # Créer une figure pour chaque paramètre
                    for param in compare_params:
                        # Préparer les données pour l'analyse
                        ideal_data_reset = ideal_data.reset_index(drop=True)
                        compare_data_reset = compare_data.reset_index(drop=True)
                        
                        # Assurer que les deux séries ont la même longueur
                        min_len = min(len(ideal_data_reset), len(compare_data_reset))
                        ideal_series = ideal_data_reset[param].iloc[:min_len]
                        compare_series = compare_data_reset[param].iloc[:min_len]
                        
                        # Calculer la différence absolue
                        diff = abs(ideal_series.values - compare_series.values)
                        
                        # Créer un index commun normalisé (0-100%)
                        x_norm = np.linspace(0, 100, min_len)
                        
                        # Créer la figure
                        fig, ax = plt.figure(figsize=(10, 6)), plt.gca()
                        
                        # Tracer les courbes de référence et de comparaison
                        ax.plot(x_norm, ideal_series, label=f"{ideal_batch} (Référence)", color='green', linewidth=2)
                        ax.plot(x_norm, compare_series, label=compare_batch, color='red', linewidth=2)
                        ax.plot(x_norm, diff, label='Différence absolue', color='orange', linestyle='--', linewidth=1)
                        
                        # Configurer le graphique
                        ax.set_title(f"Comparaison de {param} - {compare_step}")
                        ax.set_xlabel("Progression normalisée (%)")
                        ax.set_ylabel(param)
                        ax.legend()
                        plt.tight_layout()
                        
                        st.pyplot(fig)
                        
                        # Seuil de déviation
                        threshold = st.slider(f"Seuil de déviation pour {param}", 
                                           0.0, float(max(diff)*1.5), float(max(diff)*0.2),
                                           key=f"threshold_{param}")
                        
                        # Identifier les zones de déviation
                        deviation_indices = np.where(diff > threshold)[0]
                        
                        if len(deviation_indices) > 0:
                            # Grouper les indices consécutifs pour trouver les zones
                            ranges = []
                            if len(deviation_indices) > 0:
                                start = deviation_indices[0]
                                for i in range(1, len(deviation_indices)):
                                    if deviation_indices[i] != deviation_indices[i-1] + 1:
                                        ranges.append((start, deviation_indices[i-1]))
                                        start = deviation_indices[i]
                                ranges.append((start, deviation_indices[-1]))
                            
                            # Afficher les zones de déviation dans un nouveau graphique
                            fig, ax = plt.figure(figsize=(10, 6)), plt.gca()
                            
                            # Tracer les courbes
                            ax.plot(x_norm, ideal_series, label=f"{ideal_batch} (Référence)", color='green', linewidth=2)
                            ax.plot(x_norm, compare_series, label=compare_batch, color='red', linewidth=2)
                            ax.plot(x_norm, diff, label='Différence absolue', color='orange', linestyle='--', linewidth=1)
                            
                            # Ajouter une ligne horizontale pour le seuil
                            ax.axhline(y=threshold, color='r', linestyle='-', alpha=0.3, label=f'Seuil ({threshold:.2f})')
                            
                            # Surligner les zones de déviation
                            for start_idx, end_idx in ranges:
                                start_pct = x_norm[start_idx]
                                end_pct = x_norm[end_idx]
                                ax.axvspan(start_pct, end_pct, color='red', alpha=0.2)
                            
                            # Configurer le graphique
                            ax.set_title(f"Déviations détectées pour {param} - {compare_step}")
                            ax.set_xlabel("Progression normalisée (%)")
                            ax.set_ylabel(param)
                            ax.legend()
                            plt.tight_layout()
                            
                            st.pyplot(fig)
                            
                            # Résumé des déviations
                            st.warning(f"Déviations détectées pour {param}: {len(deviation_indices)} points dépassent le seuil.")
                            
                            # Analyse statistique des déviations
                            st.subheader(f"Analyse statistique des déviations pour {param}")
                            
                            col1, col2, col3 = st.columns(3)
                            with col1:
                                st.metric("Déviation maximale", f"{max(diff):.2f}")
                            with col2:
                                st.metric("Déviation moyenne", f"{np.mean(diff):.2f}")
                            with col3:
                                st.metric("% de points en déviation", f"{len(deviation_indices)/min_len*100:.1f}%")
                        else:
                            st.success(f"Aucune déviation significative détectée pour {param}.")
                else:
                    st.warning("Veuillez sélectionner au moins un paramètre à comparer.")
            else:
                st.warning("Données insuffisantes pour l'un des lots sélectionnés.")
        else:
            st.info("Veuillez sélectionner un lot de référence, un lot à comparer et une étape.")
    
    # Section de modélisation simplifiée
    st.header("Analyse Statistique")
    
    analysis_tabs = st.tabs(["Corrélations", "Analyse des tendances"])
    
    with analysis_tabs[0]:
        st.subheader("Analyse des Corrélations")
        st.markdown("""
        Cette section permet d'analyser les corrélations entre les différents paramètres
        et d'identifier les relations importantes.
        """)
        
        # Sélection des paramètres pour l'analyse de corrélation
        corr_params = st.multiselect(
            "Paramètres pour l'analyse de corrélation",
            options=[col for col in data.columns if col not in ['Batch name', 'Step', 'Time']],
            default=[col for col in data.columns if col not in ['Batch name', 'Step', 'Time']],
            key="corr_params"
        )
        
        if corr_params:
            # Filtrer les données
            corr_data = data[corr_params].dropna()
            
            # Calculer la matrice de corrélation
            corr_matrix = corr_data.corr()
            
            # Afficher la heatmap avec seaborn
            fig, ax = plt.figure(figsize=(10, 8)), plt.gca()
            sns.heatmap(corr_matrix, annot=True, cmap='RdBu_r', vmin=-1, vmax=1, ax=ax)
            ax.set_title("Matrice de Corrélation entre les Paramètres")
            plt.tight_layout()
            
            st.pyplot(fig)
            
            # Analyse des relations entre variables
            st.subheader("Relations entre Variables")
            
            col1, col2 = st.columns(2)
            with col1:
                x_var = st.selectbox("Variable X", options=corr_params, index=0, key="x_var")
            with col2:
                y_var = st.selectbox("Variable Y", options=[p for p in corr_params if p != x_var], index=0, key="y_var")
            
            # Créer un scatter plot simple
            fig, ax = plt.figure(figsize=(10, 6)), plt.gca()
            ax.scatter(data[x_var], data[y_var], alpha=0.6)
            
            # Ajouter une ligne de tendance simple
            if st.checkbox("Afficher la ligne de tendance", value=True, key="show_trendline"):
                # Calcul simple d'une régression linéaire avec numpy
                mask = ~np.isnan(data[x_var]) & ~np.isnan(data[y_var])
                x_values = data[x_var][mask].values
                y_values = data[y_var][mask].values
                
                if len(x_values) > 1 and len(y_values) > 1:
                    coeffs = np.polyfit(x_values, y_values, 1)
                    poly = np.poly1d(coeffs)
                    
                    # Tracer la ligne
                    x_range = np.array([min(x_values), max(x_values)])
                    ax.plot(x_range, poly(x_range), 'r--', 
                           label=f'y = {coeffs[0]:.4f}x + {coeffs[1]:.4f}')
                    ax.legend()
            
            ax.set_title(f"Relation entre {x_var} et {y_var}")
            ax.set_xlabel(x_var)
            ax.set_ylabel(y_var)
            plt.tight_layout()
            
            st.pyplot(fig)
            
            # Option simple d'analyse par groupe
            color_var = st.selectbox(
                "Colorer par",
                options=["Aucune coloration"] + ["Batch name", "Step"],
                index=0,
                key="color_var"
            )
            
            if color_var != "Aucune coloration":
                # Répartir par groupe
                groups = data[color_var].unique()
                
                # Créer un graphique avec des couleurs par groupe
                fig, ax = plt.figure(figsize=(10, 6)), plt.gca()
                
                for i, group in enumerate(groups):
                    group_data = data[data[color_var] == group]
                    ax.scatter(group_data[x_var], group_data[y_var], alpha=0.6, label=group)
                
                ax.set_title(f"Relation entre {x_var} et {y_var}, coloré par {color_var}")
                ax.set_xlabel(x_var)
                ax.set_ylabel(y_var)
                ax.legend(title=color_var)
                plt.tight_layout()
                
                st.pyplot(fig)
                
                # Boxplot pour comparer les distributions
                st.subheader(f"Distribution de {y_var} par {color_var}")
                
                fig, ax = plt.figure(figsize=(10, 6)), plt.gca()
                sns.boxplot(x=color_var, y=y_var, data=data, ax=ax)
                ax.set_title(f"Distribution de {y_var} par {color_var}")
                plt.tight_layout()
                
                st.pyplot(fig)
        else:
            st.warning("Veuillez sélectionner au moins un paramètre pour l'analyse.")
    
    with analysis_tabs[1]:
        st.subheader("Analyse des Tendances")
        st.markdown("""
        Cette section permet d'analyser les tendances des paramètres au fil du temps
        et d'identifier les comportements anormaux.
        """)
        
        # Sélection du paramètre à analyser
        trend_param = st.selectbox(
            "Paramètre à analyser",
            options=[col for col in data.columns if col not in ['Batch name', 'Step', 'Time']],
            index=0,
            key="trend_param"
        )
        
        # Sélection de l'étape
        trend_step = st.selectbox(
            "Étape à analyser",
            options=["Toutes les étapes"] + sorted(data['Step'].unique()),
            key="trend_step"
        )
        
        if trend_param:
            # Filtrer les données
            if trend_step == "Toutes les étapes":
                trend_data = data
            else:
                trend_data = data[data['Step'] == trend_step]
            
            # Calculer les statistiques par lot
            batch_stats = trend_data.groupby('Batch name')[trend_param].agg(['mean', 'std', 'min', 'max']).reset_index()
            
            # Afficher les statistiques
            st.subheader("Statistiques par Lot")
            st.dataframe(batch_stats)
            
            # Visualiser la distribution des moyennes
            fig, ax = plt.figure(figsize=(10, 6)), plt.gca()
            ax.bar(batch_stats['Batch name'], batch_stats['mean'], yerr=batch_stats['std'])
            ax.set_title(f"Moyenne de {trend_param} par lot" + (f" - {trend_step}" if trend_step != "Toutes les étapes" else ""))
            ax.set_xlabel("Lot")
            ax.set_ylabel(f"Moyenne de {trend_param}")
            plt.xticks(rotation=45)
            plt.tight_layout()
            
            st.pyplot(fig)
            
            # Calculer et afficher la tendance globale
            if 'Time' in trend_data.columns:
                # Réorganiser les données pour la tendance temporelle
                time_trend = trend_data.sort_values('Time')
                
                fig, ax = plt.figure(figsize=(10, 6)), plt.gca()
                
                # Tracer la tendance globale
                ax.plot(time_trend['Time'], time_trend[trend_param], 'o-', alpha=0.3)
                
                # Ajouter une ligne de tendance
                x_values = np.arange(len(time_trend))
                y_values = time_trend[trend_param].values
                
                if len(x_values) > 1 and len(y_values) > 1:
                    # Régression polynomiale pour montrer la tendance
                    degree = st.slider("Degré du polynôme pour la tendance", 1, 5, 2, key="poly_degree")
                    coeffs = np.polyfit(x_values, y_values, degree)
                    poly = np.poly1d(coeffs)
                    
                   # Tracer la ligne de tendance
                    x_smooth = np.linspace(0, len(x_values)-1, 100)
                    y_smooth = poly(x_smooth)
                    ax.plot(x_smooth, y_smooth, 'r-', linewidth=2, 
                           label=f'Tendance (degré {degree})')
                    ax.legend()
                
                ax.set_title(f"Tendance de {trend_param} au fil du temps")
                ax.set_xlabel("Temps")
                ax.set_ylabel(trend_param)
                plt.xticks(rotation=45)
                plt.tight_layout()
                
                st.pyplot(fig)
            
            # Détection simple d'anomalies
            st.subheader("Détection d'Anomalies")
            
            # Méthode simple: Identifier les valeurs au-delà d'un certain nombre d'écarts-types
            std_threshold = st.slider(
                "Nombre d'écarts-types pour détecter les anomalies", 
                1.0, 5.0, 3.0, 0.1,
                key="std_threshold"
            )
            
            # Calculer la moyenne et l'écart-type par étape
            if trend_step == "Toutes les étapes":
                # Calculer par étape
                step_stats = trend_data.groupby('Step')[trend_param].agg(['mean', 'std']).reset_index()
                
                # Créer la figure
                fig, ax = plt.figure(figsize=(10, 6)), plt.gca()
                
                # Tracer les limites pour chaque étape
                for step in trend_data['Step'].unique():
                    step_data = trend_data[trend_data['Step'] == step]
                    stats = step_stats[step_stats['Step'] == step].iloc[0]
                    
                    mean_val = stats['mean']
                    std_val = stats['std']
                    
                    upper_limit = mean_val + std_threshold * std_val
                    lower_limit = mean_val - std_threshold * std_val
                    
                    # Identifier les anomalies
                    anomalies = step_data[
                        (step_data[trend_param] > upper_limit) | 
                        (step_data[trend_param] < lower_limit)
                    ]
                    
                    # Tracer les données et les anomalies
                    ax.scatter(
                        np.arange(len(step_data)), 
                        step_data[trend_param],
                        alpha=0.3,
                        label=f"{step} - Données"
                    )
                    
                    if not anomalies.empty:
                        # Trouver les indices des anomalies dans les données originales
                        anomaly_indices = []
                        for idx, row in anomalies.iterrows():
                            anomaly_indices.append(step_data.index.get_loc(idx))
                        
                        ax.scatter(
                            anomaly_indices,
                            anomalies[trend_param],
                            color='red',
                            s=50,
                            label=f"{step} - Anomalies"
                        )
                    
                    # Tracer les limites
                    x_range = np.array([0, len(step_data)])
                    ax.plot(x_range, [upper_limit, upper_limit], 'r--', alpha=0.5)
                    ax.plot(x_range, [lower_limit, lower_limit], 'r--', alpha=0.5)
                    ax.text(x_range[1], upper_limit, f"{upper_limit:.2f}", va='bottom')
                    ax.text(x_range[1], lower_limit, f"{lower_limit:.2f}", va='top')
            else:
                # Analyser une seule étape
                mean_val = trend_data[trend_param].mean()
                std_val = trend_data[trend_param].std()
                
                upper_limit = mean_val + std_threshold * std_val
                lower_limit = mean_val - std_threshold * std_val
                
                # Identifier les anomalies
                anomalies = trend_data[
                    (trend_data[trend_param] > upper_limit) | 
                    (trend_data[trend_param] < lower_limit)
                ]
                
                # Créer la figure
                fig, ax = plt.figure(figsize=(10, 6)), plt.gca()
                
                # Tracer les données et les anomalies
                ax.scatter(
                    np.arange(len(trend_data)), 
                    trend_data[trend_param],
                    alpha=0.3,
                    label="Données"
                )
                
                if not anomalies.empty:
                    # Convertir les indices des anomalies
                    anomaly_indices = []
                    for idx in anomalies.index:
                        if idx in trend_data.index:
                            anomaly_indices.append(trend_data.index.get_loc(idx))
                    
                    ax.scatter(
                        anomaly_indices,
                        anomalies[trend_param],
                        color='red',
                        s=50,
                        label="Anomalies"
                    )
                
                # Tracer les limites
                x_range = np.array([0, len(trend_data)])
                ax.plot(x_range, [upper_limit, upper_limit], 'r--', alpha=0.5)
                ax.plot(x_range, [lower_limit, lower_limit], 'r--', alpha=0.5)
                ax.text(x_range[1], upper_limit, f"+{std_threshold} σ: {upper_limit:.2f}", va='bottom')
                ax.text(x_range[1], lower_limit, f"-{std_threshold} σ: {lower_limit:.2f}", va='top')
            
            # Configurer le graphique
            ax.set_title(f"Détection d'anomalies pour {trend_param}" + 
                        (f" - {trend_step}" if trend_step != "Toutes les étapes" else ""))
            ax.set_xlabel("Index d'observation")
            ax.set_ylabel(trend_param)
            ax.legend()
            plt.tight_layout()
            
            st.pyplot(fig)
            
            # Résumé des anomalies
            if 'anomalies' in locals() and not anomalies.empty:
                st.warning(f"{len(anomalies)} anomalies détectées sur {len(trend_data)} points ({len(anomalies)/len(trend_data)*100:.1f}%).")
                
                # Montrer les détails des anomalies
                if st.checkbox("Afficher les détails des anomalies", key="show_anomalies"):
                    st.dataframe(anomalies)
                    
                    # Option pour télécharger
                    csv = anomalies.to_csv(index=False)
                    st.download_button(
                        label="Télécharger la liste des anomalies",
                        data=csv,
                        file_name=f"anomalies_{trend_param}_{trend_step}.csv",
                        mime='text/csv',
                    )
            else:
                st.success(f"Aucune anomalie détectée pour {trend_param}" + 
                          (f" - {trend_step}" if trend_step != "Toutes les étapes" else "") +
                          f" avec un seuil de {std_threshold} écarts-types.")

    # Section d'aide à la décision
    st.header("Aide à la Décision")
    
    # Recommandations basées sur l'analyse
    st.subheader("Recommandations pour l'Amélioration des Procédés")
    st.markdown("""
    Sur la base de l'analyse des données, voici quelques recommandations pour améliorer 
    les procédés de production et réduire les déviations.
    """)
    
    # Générer des recommandations basées sur les données
    recommendations = [
        "**Surveillez étroitement les températures** pendant les phases critiques du processus, en particulier pendant la phase de réaction.",
        "**Standardisez les procédures de contrôle** pour maintenir des conditions constantes entre les lots.",
        "**Établissez des limites d'alerte** basées sur les déviations statistiques observées dans les lots historiques.",
        "**Formez les opérateurs** à reconnaître rapidement les signes de déviation et à prendre des mesures correctives.",
        "**Documentez systématiquement** toutes les interventions manuelles pendant le processus de production."
    ]
    
    for i, rec in enumerate(recommendations):
        st.markdown(f"{i+1}. {rec}")
    
    # Ajouter une section pour les notes personnalisées
    st.subheader("Notes et Observations")
    user_notes = st.text_area(
        "Ajoutez vos propres observations et recommandations",
        height=150
    )
    
    if st.button("Sauvegarder les notes"):
        st.success("Notes sauvegardées avec succès!")
        
        # Création d'un rapport combinant l'analyse et les notes
        if user_notes:
            report = f"""
            # Rapport d'Analyse des Procédés - {datetime.now().strftime("%Y-%m-%d")}
            
            ## Recommandations Système
            
            {chr(10).join([f"- {rec}" for rec in recommendations])}
            
            ## Notes et Observations
            
            {user_notes}
            """
            
            # Option pour télécharger le rapport
            st.download_button(
                label="Télécharger le rapport",
                data=report,
                file_name=f"rapport_analyse_{datetime.now().strftime('%Y%m%d')}.md",
                mime='text/markdown',
            )

# Pied de page
st.markdown("---")
st.markdown("""
**Application développée pour Sanofi** | Version 1.0  
Cette application permet d'identifier les déviations de procédés en étudiant l'évolution des paramètres de production.
""")

In [ ]:
!streamlit run app.py -- --browser.gatherUsageStats=false